In [1]:
import os
import cv2
import string
import numpy as np
from PIL import Image
from glob import glob
import matplotlib.pyplot as plt

import torch, torchvision
from torchvision import transforms

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/opt/homebrew/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LABEL = string.digits+string.ascii_uppercase
label_dict = {idx : label for idx, label in enumerate(LABEL)}
num_classes = len(label_dict)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(128, num_classes),
        )
                
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = x.view(x.size(0), -1)
        return self.fc(x)

transforms_img = transforms.Compose([
        transforms.Resize(size=(31,31)),
        transforms.CenterCrop(size=31),
        transforms.ToTensor(),
        transforms.Grayscale(3),
        transforms.Normalize(mean=(0.5,), std=(0.5,))

    ])

model = Net()
model.load_state_dict(torch.load('./models/text_recognition.ali', map_location=device))
model.to(device)

Net(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.25, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p

In [12]:
for i in glob('./images/1*.jpg'):
    image = cv2.imread(i)
    image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    image = transforms_img(image)
    image = image.view(1, 3, 31, 31).cuda() if torch.cuda.is_available() else image.view(1, 3, 31, 31)

    with torch.no_grad():
        model.eval()
        output = model(image)
        
    output = torch.nn.functional.log_softmax(output, dim=1)
    output = torch.exp(output)
    prob, top_class = torch.topk(output, k=1, dim=1) 
    print(prob, top_class)
    res_label   = label_dict[top_class.cpu().numpy()[0][0]]
    print(LABEL[top_class.cpu().numpy()[0][0]])
    res_prob    = round((prob.cpu().numpy()[0][0]), 2)
    print(res_label, res_prob, i)

tensor([[1.]]) tensor([[1]])
1
1 1.0 ./images/1_10043.jpg
tensor([[1.]]) tensor([[1]])
1
1 1.0 ./images/1_10060.jpg
tensor([[1.]]) tensor([[1]])
1
1 1.0 ./images/1_10059.jpg
tensor([[1.]]) tensor([[1]])
1
1 1.0 ./images/1_10104.jpg
tensor([[0.7827]]) tensor([[5]])
5
5 0.78 ./images/12022041104595530.jpg
tensor([[1.]]) tensor([[1]])
1
1 1.0 ./images/1_10029.jpg
tensor([[0.6902]]) tensor([[3]])
3
3 0.69 ./images/1.jpg


In [7]:
image.size()

torch.Size([1, 3, 31, 31])

In [43]:
image = cv2.imread('/Users/alimustofa/Halotec/Source Code/research/ocr/from_scratch/images/C_13549.jpg')
image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
image = transforms_img(image)
image = transforms.ToPILImage()(image)
image